In [19]:
#fungsi untuk membaca file
def read_file(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    return content

In [20]:
#fungsi untuk membaca file data latih
def get_training_set(content):
    training_set = []
    count = 0
    for row in content:
        if(row.startswith('<kalimat')):
            count += 1
        if(count <= 1000):
            training_set.append(row)
    
    return training_set

In [21]:
#fungsi untuk membaca file data uji
def get_test_set(content):
    test_set = []
    count = 0
    for row in content:
        if(row.startswith('<kalimat')):
            count += 1
        if(count > 1000 and count <= 1020):
            test_set.append(row)
    
    return test_set

In [22]:
#membaca file dari data corpus
file = read_file('Indonesian_Manually_Tagged_Corpus_ID.tsv')
train_set = get_training_set(file)
test_set = get_test_set(file)

In [23]:
#mengetahui jumlah kemunculan tag
def frekuensi(content):
    tag_count = {} #menyimpan jumlah kemunculan tag
    tag_count['<start>'] = 0 
    word_tag = {} #jumlah sebuah kata diberi tag tertentu
    tag_trans = {} #jumlah kemunculan suatu tag setelah tag tertentu
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        while not content[idx_line].startswith('</kalimat'):
            
            #Jika bukan kata pertama dalam kalimat
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
                if (content_part[0],content_part[1]) in word_tag: #keys dibiarkan menjadi terpisah 
                    word_tag[(content_part[0],content_part[1])] += 1 #untuk memudahkan ketika mengambil indeks keys dictionary
                else:    
                    word_tag[(content_part[0],content_part[1])] = 1 #dan juga agar tanda ',' yg memisahkan kata dengan tag tidak dibaca sebagai string
                    
                if is_first_word == 1: #apabila kata adalah kata pertama, maka akan muncul tag <start> sebagai penanda awal kalimat
                    current_tag_trans = ('<start>',content_part[1]) #keys dibiarkan menjadi terpisah 
                    is_first_word = 0
                else:
                    current_tag_trans = (prev_tag,content_part[1])
                    
                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]   
                
            #Jika kata pertama dalam kalimat        
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [24]:
tag_count, word_tag, tag_trans = frekuensi(train_set)

In [25]:
print(tag_count)

{'<start>': 1000, 'NN': 6245, 'SC': 1124, 'VB': 2758, 'NNP': 3715, 'JJ': 1033, 'RB': 546, 'IN': 2167, 'Z': 2653, 'CD': 2276, 'CC': 776, 'PR': 491, 'PRP': 691, 'MD': 522, 'FW': 397, 'NEG': 123, 'DT': 20, 'NND': 148, 'SYM': 299, 'RP': 18, 'WH': 8, 'OD': 93, 'X': 14, 'UH': 4}


In [26]:
#mengetahui nilai dari probabilitas perpindahan antar tag satu sama lain dari tabel transisis
def dict_prob_transition(tag_trans,tag_count): 
    probability_trans = {}
    keys = tag_trans.keys()
    for tag in keys:
        if (tag[0],tag[1]) in tag_trans: 
            probability_trans[(tag[0],tag[1])] = tag_trans[(tag[0],tag[1])]/tag_count[tag[0]]
    return probability_trans

In [27]:
probability_trans = dict_prob_transition(tag_trans,tag_count) #membangun dictionary untuk probabilitas transition tag pada data train
print(probability_trans)

{('<start>', 'NN'): 0.313, ('NN', 'SC'): 0.06693354683746998, ('SC', 'VB'): 0.4279359430604982, ('VB', 'NN'): 0.3299492385786802, ('<start>', 'NNP'): 0.162, ('NNP', 'NNP'): 0.5055181695827725, ('NNP', 'VB'): 0.06944818304172275, ('NN', 'JJ'): 0.07638110488390712, ('JJ', 'SC'): 0.08518877057115198, ('VB', 'RB'): 0.022117476432197244, ('RB', 'JJ'): 0.24725274725274726, ('JJ', 'IN'): 0.1403678606001936, ('IN', 'NN'): 0.5652976465159206, ('NN', 'NNP'): 0.07910328262610088, ('NNP', 'Z'): 0.19138627187079407, ('<start>', 'CD'): 0.026, ('CD', 'NN'): 0.18541300527240773, ('NN', 'VB'): 0.04547638110488391, ('RB', 'CD'): 0.07142857142857142, ('VB', 'IN'): 0.1649746192893401, ('NN', 'NN'): 0.32602081665332266, ('NN', 'CC'): 0.03698959167333867, ('CC', 'NN'): 0.4110824742268041, ('NN', 'IN'): 0.08310648518815052, ('IN', 'NNP'): 0.19520073834794646, ('VB', 'CD'): 0.09028281363306744, ('CC', 'VB'): 0.10438144329896908, ('VB', 'VB'): 0.049673676577229875, ('CD', 'Z'): 0.18321616871704746, ('Z', 'NN')

In [28]:
#menegtahui probabilitas dari emisi 
def dict_prob_emission(word_tag,tag_count):
    probability_emission = {}
    word_key = []
    keys = word_tag.keys()
    for part_key in keys:
        if((part_key[0],part_key[1])) in word_tag:
            probability_emission[(part_key[0],part_key[1])] = word_tag[(part_key[0],part_key[1])]/tag_count[(part_key[1])]
    return probability_emission

In [29]:
probability_emission = dict_prob_emission(word_tag,tag_count) #membangun dictionary untuk probabilitas transition tag pada data train
print(probability_emission)

{('Kera', 'NN'): 0.0001601281024819856, ('untuk', 'SC'): 0.10854092526690391, ('amankan', 'VB'): 0.00036258158085569254, ('pesta olahraga', 'NN'): 0.0001601281024819856, ('Pemerintah', 'NNP'): 0.0005383580080753701, ('kota', 'NNP'): 0.0008075370121130552, ('Delhi', 'NNP'): 0.0013458950201884253, ('mengerahkan', 'VB'): 0.00036258158085569254, ('monyet', 'NN'): 0.001921537229783827, ('mengusir', 'VB'): 0.0014503263234227702, ('monyet-monyet', 'NN'): 0.0003202562049639712, ('lain', 'JJ'): 0.028073572120038724, ('yang', 'SC'): 0.45640569395017794, ('berbadan', 'VB'): 0.0007251631617113851, ('lebih', 'RB'): 0.1446886446886447, ('kecil', 'JJ'): 0.013552758954501452, ('dari', 'IN'): 0.10844485463774804, ('arena', 'NN'): 0.0003202562049639712, ('Pesta Olahraga', 'NNP'): 0.00026917900403768504, ('Persemakmuran', 'NNP'): 0.00026917900403768504, ('.', 'Z'): 0.36977007161703734, ('Beberapa', 'CD'): 0.0004393673110720562, ('laporan', 'NN'): 0.0022417934347477983, ('menyebutkan', 'VB'): 0.0036258158

In [30]:
#menampung kata dan post tag
sentences= []
tags = []
index = 0
while index < len(test_set):
    sent = [] #untuk menampung katanya
    tag = [] #untuk menampung postag dari tiap kata
    while not test_set[index].startswith('</kalimat'): #batas akhir dari kalimat
        if  not test_set[index].startswith('<kalimat'):
            content_part = test_set[index].split('\t')
            sent.append(content_part[0])
            tag.append(content_part[1])
        index = index + 1
    sentences.append(sent)
    tags.append(tag)
    index = index+2

In [31]:
#fungsi yang mencari nilai viterbi 
def viterbi(probability_trans,probability_emission,sentence):
    tag_sequence = {}
    viterbi_mat = {}
    emission_table = {}
    transition_table = {}
    word_emission = []
    tag_array = []
    temp = {}                                                                    
    
    # Membuat Table Emission dari data tes
    sentence_words = sentence
    keys_emission = probability_emission.keys()
    
    for keys in keys_emission: 
        word_emission.append(keys[0]) #mengambil seluruh kata yang ada didalam model 

    for i in range(0,len(sentence_words)): #looping kata dalam kalimat
        if sentence_words[i] not in word_emission: #Jika ada kata dalam data test yang belum pernah terbaca di dalam model
            if i == 0: #jika kata tersebut adalah kata pertama
                for tag2 in tag_count.keys(): #maka tag yang diberikan adalah seluruh tag yang ada didalam model
                    temp[(sentence_words[0],tag2)] = 1 / tag_count[tag2] 
            else: #jika kata tersebut bukan kata pertama
                for tag in tag_array: #maka tag yang diberikan adalah hanya tag yang pernah muncul dalam kumpulan tag dari asal kalimat kata tersebut
                    temp[(sentence_words[i],tag)] = 1 / tag_count[tag]

        probability_emission.update(temp) #mengisi seluruh keys dan values tersebut kedalam model agar selanjutnya dapat dibaca dalam model jika ada kata yang sama
        keys_emission = probability_emission.keys() #update keys dari dictionary jumlah kemunculan kata
        for keys in keys_emission:
            word_emission.append(keys[0]) #update array yang menampung seluruh keys dari dictionary jumlah kemunculan kata

        if sentence_words[i] in word_emission: #Jika kata sudah ada didalam model yang dibangun
            for key in keys_emission:
                if (sentence_words[i],key[1]) in keys_emission: #mencoba seluruh kombinasi kemunculan kata dengan seluruh tag yang ada di dalam dictionary emission
                    emission_table[(sentence_words[i],key[1])] = probability_emission[(key[0],key[1])] #jika ada, diambil nilai probabilitynya dan digunakan untuk tabel emission data test
                else:
                    emission_table[(sentence_words[i],key[1])] = 0 #jika tidak ada, maka tabel emission untuk kombinasi kata dengan tag tersebut adalah 0               
                if sentence_words[i] == key[0]:
                    tag_array.append(key[1]) #mengambil seluruh tag yang memiliki nilai probabilitas pada tabel emission

    # print(emission_table)
    # print(word_emission)
                                                                         # Membuat Table Transition dari data test
    for tag1 in tag_array: #melihat seluruh kemungkinan kombinasi tag yang memiliki nilai pada tabel emission data test
        for tag2 in tag_array:
            if ('<start>',tag1) in probability_trans.keys(): #untuk kombinasi tag start dengan seluruh tag yang lain
                transition_table[('<start>',tag1)] = probability_trans[('<start>',tag1)]
            elif ('<start>',tag1) not in probability_trans.keys():
                transition_table[('<start>',tag1)] = 0 #jika tidak ada kombinasinya maka nilai transitionnya adalah 0
            if (tag1,tag2) in probability_trans.keys(): #untuk kombinasi kemungkinan suatu tag terhadap tag yang lain
                transition_table[(tag1,tag2)] = probability_trans[(tag1,tag2)]
            elif (tag1,tag2) not in probability_trans.keys():
                transition_table[(tag1,tag2)] = 0 #jika tidak ada maka nilai transitionnya 0
    # print(transition_table)
    # print(tag_array)

    # HMM-Viterbi
    for i in range(0,len(sentence_words)):
            if i == 0:
                for tag1 in tag_array: #Jika kata adalah kata pertama, maka tidak perlu dikalikan dengan nilai viterbi, karena nilai viterbi belum ada
                    viterbi_mat[(sentence_words[i],tag1)] = emission_table[(sentence_words[i],tag1)] * transition_table[('<start>',tag1)]
                    tag_sequence[(sentence_words[i],tag1)] = tag1 #untuk melakukan tracking jalur tag 
            else:
                for tag1 in tag_array:
                    new_score = {} #menampung score perhitungan probabilitas tag sequence yang terjadi
                    for tag2 in tag_array:
                        new_score[(sentence_words[i-1],tag2)] = viterbi_mat[(sentence_words[i-1],tag2)] * transition_table[(tag2,tag1)] 
                    max_path = max(new_score, key=lambda k: new_score[k]) #mencari nilai maksimum dari new score yang dihitung
                    viterbi_mat[(sentence_words[i],tag1)] = new_score[max_path] * emission_table[(sentence_words[i],tag1)]  #mengisi nilai probability tag sequence dengan nilai probability yang paling tinggi
                    tag_sequence[(sentence_words[i],tag1)] = max_path #mengambil tag yang menghasilkan nilai probability tertinggi

    #Mencari nilai paling maksimum dari seluruh nilai probability tag sequence yang didapat
    kata_terakhir = sentence_words[len(sentence_words)-1]
    tagg = tag_array[0]
    tags = [tagg]
    maksimum = viterbi_mat[(kata_terakhir,tagg)]
    for i in range(1,len(tag_array)):
        if viterbi_mat[(kata_terakhir,tag_array[i])] > maksimum:
            maksimum = viterbi_mat[(kata_terakhir,tag_array[i])]
            tags = [tag_array[i]]
    for i in range(len(sentence_words)-1,0,-1):
        tags.insert(0,tag_sequence[(sentence_words[i],tags[0])][1])

    return tags

In [32]:
#menghitung viterbi
hasil_viterbi = []
counter=0
for sentence in sentences:
    viterbi_sentence = viterbi(probability_trans,probability_emission,sentence)
    hasil_viterbi.append(viterbi_sentence) #mengambil hasil tag yang diperoleh dari viterbi

kumpulan_tag_viterbi = []
for tag in hasil_viterbi:
    for i in range(len(tag)): 
        kumpulan_tag_viterbi.append(tag[i]) #mengambil seluruh tag yang ada didalam array hasil viterbi
print(len(kumpulan_tag_viterbi))

680


In [33]:
kumpulan_tag_viterbi

['NNP',
 'NNP',
 'VB',
 'NNP',
 'NN',
 'NN',
 'SYM',
 'IN',
 'NNP',
 'NNP',
 'NNP',
 'NNP',
 'CC',
 'FW',
 'NN',
 'OD',
 'NN',
 'CD',
 'JJ',
 'SYM',
 'CD',
 'CD',
 'SC',
 'MD',
 'VB',
 'IN',
 'NNP',
 'CD',
 'IN',
 'NN',
 'NN',
 'PR',
 'NNP',
 'Z',
 'VB',
 'NNP',
 'IN',
 'NN',
 'PRP',
 'IN',
 'NNP',
 'Z',
 'NNP',
 'Z',
 'IN',
 'NN',
 'Z',
 'NN',
 'PR',
 'VB',
 'IN',
 'NN',
 'NN',
 'NN',
 'SC',
 'JJ',
 'IN',
 'NN',
 'NN',
 'NN',
 'Z',
 'NN',
 'NN',
 'SC',
 'VB',
 'IN',
 'NN',
 'IN',
 'NN',
 'IN',
 'NN',
 'CC',
 'DT',
 'NN',
 'SC',
 'RB',
 'Z',
 'PR',
 'Z',
 'VB',
 'NN',
 'PR',
 'Z',
 'NN',
 'PR',
 'VB',
 'IN',
 'NN',
 'FW',
 'NN',
 'SC',
 'JJ',
 'CC',
 'JJ',
 'PRP',
 'NN',
 'IN',
 'NN',
 'NN',
 'PR',
 'Z',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 'NND',
 'IN',
 'NNP',
 'NNP',
 'NNP',
 'NNP',
 'NNP',
 'IN',
 'NN',
 'NN',
 'SYM',
 'CD',
 'Z',
 'CC',
 'NN',
 'PRP',
 'SYM',
 'CD',
 'NND',
 'IN',
 'NNP',
 'NNP',
 

In [34]:
#kumpulan kata dan tag yang dari sebuah kalimat 
kumpulan_kata = []
kumpulan_tag = []
for sentence in sentences:
    for i in range(len(sentence)): 
        kumpulan_kata.append(sentence[i])

for tag in tags:
    for i in range(len(tag)): 
        kumpulan_tag.append(tag[i])

In [35]:
kumpulan_tag

['NNP',
 'NNP',
 'NNP',
 'NNP',
 'VB',
 'NN',
 'SYM',
 'IN',
 'NNP',
 'NNP',
 'NNP',
 'NNP',
 'CC',
 'FW',
 'NNP',
 'NNP',
 'NN',
 'CD',
 'JJ',
 'SYM',
 'CD',
 'CD',
 'SC',
 'MD',
 'VB',
 'IN',
 'NNP',
 'CD',
 'SC',
 'NN',
 'NN',
 'PR',
 'JJ',
 'Z',
 'VB',
 'NNP',
 'IN',
 'NN',
 'PRP',
 'IN',
 'NNP',
 'Z',
 'NNP',
 'Z',
 'IN',
 'NNP',
 'Z',
 'NN',
 'PR',
 'VB',
 'IN',
 'NN',
 'NN',
 'NNP',
 'SC',
 'JJ',
 'IN',
 'NN',
 'NN',
 'NN',
 'Z',
 'NN',
 'NN',
 'SC',
 'VB',
 'IN',
 'NN',
 'FW',
 'NN',
 'NN',
 'NN',
 'CC',
 'NN',
 'NN',
 'SC',
 'RB',
 'Z',
 'CC',
 'Z',
 'VB',
 'NN',
 'PR',
 'Z',
 'NN',
 'PR',
 'VB',
 'IN',
 'NN',
 'FW',
 'NN',
 'SC',
 'JJ',
 'CC',
 'JJ',
 'PRP',
 'NN',
 'IN',
 'NN',
 'NN',
 'NN',
 'Z',
 'NN',
 'NN',
 'NNP',
 'IN',
 'CD',
 'NNP',
 'CD',
 'Z',
 'NN',
 'MD',
 'VB',
 'IN',
 'NNP',
 'NNP',
 'NNP',
 'NNP',
 'NNP',
 'IN',
 'NN',
 'NN',
 'CD',
 'CD',
 'Z',
 'CC',
 'NN',
 'PRP',
 'CD',
 'CD',
 'VB',
 'IN',
 'NNP',
 'NNP',
 'NNP',
 'Z',
 'NNP',
 'NNP',
 'NNP',
 'NNP',
 'Z'

In [36]:
#Menghitung akurasi data
counter=0
jumlah_salah = 0
for i in range(len(kumpulan_tag_viterbi)):
    if kumpulan_tag_viterbi[i] == kumpulan_tag[i]:
        counter += 1
    elif kumpulan_tag_viterbi[i] != kumpulan_tag[i]:
        jumlah_salah += 1
        print(str(jumlah_salah)+'.' , 'Kata: '+ kumpulan_kata[i], '=> Tag Test: '+kumpulan_tag_viterbi[i], 'Tag Asli: '+kumpulan_tag[i])


1. Kata: Indonesia => Tag Test: VB Tag Asli: NNP
2. Kata: menegaskan => Tag Test: NN Tag Asli: VB
3. Kata: Obligasi => Tag Test: NN Tag Asli: NNP
4. Kata: I => Tag Test: OD Tag Asli: NNP
5. Kata: dengan => Tag Test: IN Tag Asli: SC
6. Kata: stabil => Tag Test: NNP Tag Asli: JJ
7. Kata: Pefindo => Tag Test: NN Tag Asli: NNP
8. Kata: SERA => Tag Test: NN Tag Asli: NNP
9. Kata: leasing => Tag Test: IN Tag Asli: FW
10. Kata: guna => Tag Test: IN Tag Asli: NN
11. Kata: proteksi => Tag Test: DT Tag Asli: NN
12. Kata: Namun => Tag Test: PR Tag Asli: CC
13. Kata: bekas => Tag Test: PR Tag Asli: NN
14. Kata: Komposisi => Tag Test: <start> Tag Asli: NN
15. Kata: pemegang saham => Tag Test: <start> Tag Asli: NN
16. Kata: SERA => Tag Test: <start> Tag Asli: NNP
17. Kata: per => Tag Test: <start> Tag Asli: IN
18. Kata: 30 => Tag Test: <start> Tag Asli: CD
19. Kata: Juni => Tag Test: <start> Tag Asli: NNP
20. Kata: 2007 => Tag Test: <start> Tag Asli: CD
21. Kata: , => Tag Test: <start> Tag Asli: Z
2

In [38]:
akurasi = (counter/len(kumpulan_tag)) * 100
print('Akurasi: ',akurasi, '%')

Akurasi:  75.58823529411765 %
